In [ ]:
import os
import sys
import gym
import json
import time
import pickle
import pandas as
import numpy as np
from itertools import groupby
from datetime import datetime, timedelta
import argparse
from sqlite3 import Timestamp
from utils import make_dict

In [ ]:
def main():
    # Initialize the simulation environment
    env = gym.make('Lab-Building-control-v0')
    publisher = pub.MyPublisher('Lab-Building-control')
    publisher.start()

    # Variables configuration
    obs_name = [
        "T_ext", "Wind_Speed", "Wind_Direction", "Reception",
        "Bathroom1", "Bathroom2", "Bathroom3", "Corridor", "Office", 
        "Electricity", "DistrictHeating", "DistrictCooling", "Bath1_shade", 
        "Classroom1_shade1", "Classroom1_shade2", "Bath_shade", "Office_shade1", "Office_shade2", 
        "Classroom2_shade1", "Classroom2_shade2"
    ]
    tot_days = 14
    n_step_hour = 4
    n_step = n_step_hour * 24

    # Simulation loop
    for i_episode in range(tot_days):
        time_step, observations, is_terminal, _ = env.reset()
        start_time = datetime(year=env.start_year, month=env.start_month, day=env.start_day)
        cur_time = start_time
        obs_dict = make_dict(obs_name, observations)
        send_data(publisher, obs_dict, cur_time)

        actuator = act.MyActuator('Actuator1', 'actuator')
        actuator.run()

        for t in range(n_step):
            # Check actuator status and execute step
            if not actuator.wait:
                action_step = json.loads(actuator.msg.payload)
                actuator.wait = True
                input_actions = [float(val) for val in action_step.values()]
                
                if input-actions:
                    time_step, observations, is_terminal, _ = env.step(input_actions)
                    cur_time += timedelta(seconds=time_step)
                    obs_dict = make_dict(obs_name, observations)
                    send_data(publisher, obs_dict, cur_time)

            time.sleep(2)  # Delay to simulate time between actions

    env.end_env()

In [ ]:
def send_data(publisher, obs_dict, cur_time):
    """Helper function to send observation data."""
    for key, val in obs_dict.items():
        payload = {
            "location": "MyBuilding",
            "measurement": "test",
            "node": key,
            "time_stamp": str(cur_time),
            "value": str(val)
        }
        publisher.myPublish("/ict4bd", json.dumps(payload))
        time.sleep(0.01)  # Small delay between sending data

In [ ]:
if __name__ == '__main__':
    main()